In [30]:
import pandas as pd

In [31]:
df = pd.read_csv('C:\Ali\Learning\Data Science\SE Factory\csvs\WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [32]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [33]:
for col in df.columns:
    temp = df[col].unique()
    if len(temp) > 10:
        temp = len(temp)
    print(f'{col}: {temp}')

customerID: 7043
gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
tenure: 73
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: 1585
TotalCharges: 6531
Churn: ['No' 'Yes']


so here we got unique values for each factor or number of unique value if they are a lot.  
In this way we can take a glance on our factors to understand their meaning  
  
The data set describes customers of a telco corp, by several demographical variables (gender, partner, seniorcitizen, dependents), account information (tenure, contract, payment, billing) and some services that customer may have purchsed (internet, streaming, support, backup...)

All those variables are descriptive, and the target column is churn column, which is boolean column that specifies whether a customer left or not.

we're gonna try to analyze the different attributes and relate any to the churn attribute, to conclude the main reasons customers left and enable the telco company to predict which customersmay leave in order to try avoiding it


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Fortunately we don't have any null values

##### just to make life easier I'm  gonna do some simple changes

lower column names

In [35]:
df.rename(columns={x:x.lower() for x in df.columns}, inplace=True)

change some Yes/No variables to bool

In [36]:
yesNo_columns = ['churn','paperlessbilling','dependents','partner']
for col in yesNo_columns:
    df[col] = df[col] == 'Yes'

change total charges column to float

In [37]:
df['totalcharges'] = df.apply(lambda x: 0 if x.totalcharges==' ' else x.totalcharges, axis=1)
df['totalcharges'] = df['totalcharges'].astype('float')

First of all, how many customers churned?

In [38]:
df.churn.sum()

1869

## Demographical variables

In [39]:
df_grouped = df[['customerid','seniorcitizen','partner','dependents','churn']].groupby('partner').agg({'seniorcitizen':'sum','dependents':'sum','customerid':'count','churn':'sum'})
df_grouped[['seniorcitizen','churn','dependents']] = df_grouped[['seniorcitizen','churn','dependents']].apply(lambda x: (x/(df_grouped.customerid)).round(3)*100, axis=0)
df_grouped

,seniorcitizen,dependents,customerid,churn
partner,,,,
False,15.6,9.9,3641,33.0
True,16.8,51.4,3402,19.7


1/3 of customers without partners leave, while 20% of customers with partners.  
this shows that having a partner is a positive factor that helps avoid customer churn

In [40]:
df_grouped = df[['customerid','seniorcitizen','partner','dependents','churn']].groupby('dependents').agg({'seniorcitizen':'sum','partner':'sum','customerid':'count','churn':'sum'})
df_grouped[['seniorcitizen','churn','partner']] = df_grouped[['seniorcitizen','churn','partner']].apply(lambda x: (x/(df_grouped.customerid)).round(3)*100, axis=0)
df_grouped

,seniorcitizen,partner,customerid,churn
dependents,,,,
False,21.3,33.5,4933,31.3
True,4.3,82.9,2110,15.5


Having dependents is also a positive factor, when a customer has dependets, it is less likely to churn

and as this table and the previous one show, partner and dependents are correlated factors that usually coexist

In [41]:
df_grouped = df[['customerid','seniorcitizen','partner','dependents','churn']].groupby('seniorcitizen').agg({'partner':'sum','dependents':'sum','customerid':'count','churn':'sum'})
df_grouped[['partner','churn','dependents']] = df_grouped[['partner','churn','dependents']].apply(lambda x: (x/(df_grouped.customerid)).round(3)*100, axis=0)
df_grouped

,partner,dependents,customerid,churn
seniorcitizen,,,,
0,47.9,34.2,5901,23.6
1,50.2,8.0,1142,41.7


being a senior citizen, the chance of churning is approximately double.

This is maybe related to the fact that only 8% of seniors citizens have dependents, which as stated before is a positive factor

or maybe it is related to the type of services or account details that seniors tend to choose, lets see

## Account Details

### Tenure

In [42]:
bins = [df.tenure.min() + i*12 for i in range(7)]
labels = [f'{i}-{i+1} years' for i in range(6)]
labels[-1] = '5+ years'
df['tenure_group'] = pd.cut(df['tenure'], bins=bins, labels=labels)

In [43]:
df_grouped = df[['tenure_group','customerid','churn']].groupby('tenure_group').agg({'customerid':'count','churn':'sum'})
df_grouped['percentage'] = df_grouped.apply(lambda x: (x.churn/x.customerid).round(3)*100, axis=1)
df_grouped

,customerid,churn,percentage
tenure_group,,,
0-1 years,2175,1037,47.7
1-2 years,1024,294,28.7
2-3 years,832,180,21.6
3-4 years,762,145,19.0
4-5 years,832,120,14.4
5+ years,1407,93,6.6


As expected, tenure is most effecting factor on churn, since customers who have been here for a long time are consistent, loyal, and trust was built between both sides over time. we can say that as tenure increase, the chance that the customer leaves decreases.

Most customers are either less than 1 year or more than 5 years customers, which shows that it's good that so much customers are loyal, but also highlights the chances to improve and increase customers

### Contract

In [44]:
piv = df.pivot_table(index='contract',columns='paperlessbilling', values=['churn','customerid'], aggfunc={'churn':'sum','customerid':'count'}, margins=True, margins_name='Total')
piv

churn             customerid            
paperlessbilling False  True Total      False  True Total
contract                                                 
Month-to-month     406  1249  1655       1289  2586  3875
One year            48   118   166        673   800  1473
Two year            15    33    48        910   785  1695
Total              469  1400  1869       2872  4171  7043

paperless billing is easier and faster, for month-to-month contracts' customers, it's the preferred choice  
Also, it is less common for customers who don't use paperlessbilling to churn, as higher percentage of them tend to go with 1 or 2 years contracts

it is not that paperlessbilling is a negative factor that we should avoid, but it's the easier choice for customers who don't have the intention to stay forever

we can see that most customers who churn are those who prefered month-to-month contract, since it is short-term, needs less commitment and good for trying it for the first time  
That is a logical thing as customers who subscribe in long term services and sign 1+ years contracts usually build better relation with the company and end up with 4+ years tenure

lets test this conclusion by grouping by contract and getting their distribution over tenure groups

In [45]:
df_grouped = df[['contract','tenure_group']].groupby('contract')['tenure_group'].value_counts(normalize=True).mul(100).round(2)
df_grouped

contract        tenure_group
Month-to-month  0-1 years       51.46
                1-2 years       19.02
                2-3 years       12.54
                3-4 years        8.15
                4-5 years        6.04
                5+ years         2.79
One year        4-5 years       21.81
                5+ years        21.26
                3-4 years       18.21
                2-3 years       16.98
                1-2 years       13.38
                0-1 years        8.36
Two year        5+ years        58.52
                4-5 years       16.44
                3-4 years       10.56
                2-3 years        5.70
                1-2 years        5.34
                0-1 years        3.44
Name: tenure_group, dtype: float64

43% of customer with 1 year contract have 4+ years tenure
75% of customers with 2-year contract have 4+ years tenure

so to wrap it up, as the length of the contract increase, the longer the tenure is

### Payment method

In [46]:
df_grouped = df.groupby('paymentmethod').agg({'totalcharges':'sum','churn':'sum','customerid':'count'}).rename(columns={'customerid':'count'})
df_grouped['churn_percentage'] = df_grouped.apply(lambda x: (x.churn/x['count']).round(3)*100, axis=1)
df_grouped['mean_charge_per_customer'] = df_grouped.apply(lambda x: (x.totalcharges/x['count']).round(-1), axis=1)
df_grouped.sort_values(by=['churn_percentage'],ascending=False,inplace=True)
df_grouped

,totalcharges,churn,count,churn_percentage,mean_charge_per_customer
paymentmethod,,,,,
Electronic check,4944903.25,1071,2365,45.3,2090.0
Mailed check,1691392.20,308,1612,19.1,1050.0
Bank transfer (automatic),4748279.90,258,1544,16.7,3080.0
Credit card (automatic),4671593.35,232,1522,15.2,3070.0


Around 1/3 customers tend to pay using electronic check, and the highest percentage of churning is from this group, at 45%

Note that mailed check is used for relatively small charges, compared to other payment methods

## Services

This is the most important, since customer retention is all about the satisfaction of our customers by our products quality and price,  
we have few questions we must answer:
* what services are crucial for customer retention?
* is there a service that is leading some customers to churn for some reason?

lets first check the most purchsed services:

In [47]:
services = ['phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
    'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
    'streamingmovies']
purchased_services = df[services].apply(lambda x:x.str[:2]!='No', axis=0)
purchased_services = purchased_services.sum().sort_values(ascending=False).to_frame().rename(columns={0:'subscribers'})
purchased_services['percentage'] = purchased_services['subscribers'].divide(df.shape[0]).mul(100).round(2)
purchased_services

,subscribers,percentage
phoneservice,6361,90.32
internetservice,5517,78.33
multiplelines,2971,42.18
streamingmovies,2732,38.79
streamingtv,2707,38.44
onlinebackup,2429,34.49
deviceprotection,2422,34.39
techsupport,2044,29.02
onlinesecurity,2019,28.67


Almost 90% of customers purchased the phone service -half of them have multiple lines- and about 80% subscribed in internet service. Other services have between 28% and 38% of customers subscribed

However, to be accurate, we should take into consideration the fact that most services are only offered for those subscribed in internet service, so we're gonna narrow our total customers here to them

In [48]:
internet_services = ['internetservice', 'onlinesecurity',
    'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
    'streamingmovies']
internet_purchsed_services = purchased_services.loc[internet_services].sort_values(by='subscribers',ascending=False)
internet_purchsed_services['percentage'] = (internet_purchsed_services['subscribers']/(purchased_services.loc['internetservice','subscribers'])).mul(100).round(2)
internet_purchsed_services

,subscribers,percentage
internetservice,5517,100.00
streamingmovies,2732,49.52
streamingtv,2707,49.07
onlinebackup,2429,44.03
deviceprotection,2422,43.90
techsupport,2044,37.05
onlinesecurity,2019,36.60


Now things are more clear, so around 50% of customers subscribed in internet service are also subscribed to streaming services (TV or movies)

In [49]:
internet_purchsed_services = df.loc[df.internetservice!='No',internet_services].groupby('internetservice').agg(lambda x: (x!='No').sum())
# add count churns
internet_purchsed_services['churn'] = [df.loc[df.internetservice==i, 'churn'].sum() for i in internet_purchsed_services.index]

# add total subscribers
internet_purchsed_services['total_subscribers'] = [df.loc[df.internetservice==i, 'churn'].count() for i in internet_purchsed_services.index]

# rearrange order of columns
cols = internet_purchsed_services.columns.to_list()
cols[-2],cols[-1] = cols[-1], cols[-2]
internet_purchsed_services = internet_purchsed_services[cols]

# add percentage col
internet_purchsed_services['churn_percentage'] = (internet_purchsed_services['churn']/internet_purchsed_services['total_subscribers']).mul(100).round(2)

# add ratio row
internet_purchsed_services.loc['Fiber/DSL ratio',:] = internet_purchsed_services.apply(lambda x: x.loc['Fiber optic']/x.loc['DSL']).round(2)

internet_purchsed_services

,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,total_subscribers,churn,churn_percentage
internetservice,,,,,,,,,
DSL,1180.00,1086.00,1065.00,1178.00,957.00,981.00,2421.00,459.00,18.96
Fiber optic,839.00,1343.00,1357.00,866.00,1750.00,1751.00,3096.00,1297.00,41.89
Fiber/DSL ratio,0.71,1.24,1.27,0.74,1.83,1.78,1.28,2.83,2.21


More people subscribe to fiber optic internet service than DSL, however, the percentage of churning within those more than double others!  
Further investigation should be done about customers' feedback on fiber optic internet service through a survey for example, to check if there is any problem in this very important service that is causing customers to churn

For sub-internet-services, it is more frequent option for subscribers in Fiber optic internet rather than DSL, except for online security and tech support services, where they seem less interested in

can this carelessness about security and support be the cause of leaving later?

In [50]:
# create dataframe showing difference in number of churns for each service
churn_by_service = []
for service in internet_services[1:]:
    a = pd.Series(df.loc[df[service]!='No internet service', ['churn',service]].groupby(service).sum()['churn'], name=service)
    churn_by_service.append(a)
churn_by_service = pd.concat(churn_by_service, axis=1)
churn_by_service.rename({'No':'unsubscribed churns', 'Yes':'subscribed churns'}, inplace=True)
churn_by_service

,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies
unsubscribed churns,1461,1233,1211,1446,942,938
subscribed churns,295,523,545,310,814,818


and below we can see those values as percentages:

In [51]:
# create dataframe showing difference in number of churns for each service
churn_by_service = []
percent = lambda x: (x.sum()/x.count()).round(4)*100
for service in internet_services[1:]:
    a = df.loc[df[service]!='No internet service', ['churn',service]].groupby(service).agg(percent)
    a = pd.Series(a['churn'], name=service)
    churn_by_service.append(a)
churn_by_service = pd.concat(churn_by_service, axis=1)
churn_by_service.rename({'No':'unsubscribed churn percentage', 'Yes':'subscribed churns percentage'}, inplace=True)
churn_by_service

,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies
unsubscribed churn percentage,41.77,39.93,39.13,41.64,33.52,33.68
subscribed churns percentage,14.61,21.53,22.50,15.17,30.07,29.94


we can reach the same results in a different shape using slightly different way:

In [52]:
melted_df = df.loc[df.internetservice!='No', ['customerid']+internet_services[1:]+['monthlycharges','churn']]
melted_df = melted_df.melt(id_vars=['customerid','monthlycharges','churn'], var_name='service', value_name='is_subscribed')
melted_df[['churn','service','is_subscribed']].groupby(['service','is_subscribed']).mean()

churn
service          is_subscribed          
deviceprotection No             0.391276
                 Yes            0.225021
onlinebackup     No             0.399288
                 Yes            0.215315
onlinesecurity   No             0.417667
                 Yes            0.146112
streamingmovies  No             0.336804
                 Yes            0.299414
streamingtv      No             0.335231
                 Yes            0.300702
techsupport      No             0.416355
                 Yes            0.151663

this table shows for every service, how much subscribed customers leaved and how much non-subscribers leaved, to find whether each of those services presence has an effect on churning

for streaming services, noticed that leaving customers tend to subscribe to them more than other services

However, for maintenance and extra services like security, protection, backup, and support, most who leave are from those not subscribed, so we can confidently conclude that these services has a major role in customer satisfaction and thus customer retention

so contnuing the previous insight, we can say yes, security and support services are important and the tendency of fiber optic internet subscribers to avoid those services maybe the cause of increase of churns in this category of customers

is the need for tech support related to senior citizens?

In [53]:
df_grouped = df[['customerid','seniorcitizen','techsupport','churn']].groupby(['seniorcitizen','techsupport']).agg({'customerid':'count','churn':'sum'})
df_grouped[['churn']] = df_grouped[['churn']].apply(lambda x: (x/(df_grouped.customerid)).round(3)*100, axis=0)
df_grouped

customerid  churn
seniorcitizen techsupport                           
0             No                         2643   38.8
              No internet service        1474    7.3
              Yes                        1784   14.5
1             No                          830   50.6
              No internet service          52    9.6
              Yes                         260   19.6

In [54]:
df_grouped = df[['customerid','seniorcitizen','onlinesecurity','churn']].groupby(['seniorcitizen','onlinesecurity']).agg({'customerid':'count','churn':'sum'})
df_grouped[['churn']] = df_grouped[['churn']].apply(lambda x: (x/(df_grouped.customerid)).round(3)*100, axis=0)
df_grouped

customerid  churn
seniorcitizen onlinesecurity                        
0             No                         2690   39.2
              No internet service        1474    7.3
              Yes                        1737   13.3
1             No                          808   50.4
              No internet service          52    9.6
              Yes                         282   22.7

yes, 50% of seniors subscribed in internet service without tech support churn, very high percentage!

## Charges

In [55]:
df[['monthlycharges','churn']].groupby('churn').mean()

,monthlycharges
churn,
False,61.265124
True,74.441332


Customers who churn have by average higher monthly charges, and this intersects with what we've found before, that most customers who left had month-to-month contracts, which logically charge a bit higher charge than long-term contracts

In [56]:
df[['monthlycharges','streamingmovies']].groupby('streamingmovies').mean()

,monthlycharges
streamingmovies,
No,65.434147
No internet service,21.079194
Yes,88.475714


In [57]:
df[['monthlycharges','streamingtv']].groupby('streamingtv').mean()

,monthlycharges
streamingtv,
No,65.387687
No internet service,21.079194
Yes,88.736738


second cause why customers leaving have higher charge average; is that a larger percentage of them subscribe in streaming services (stated in previous part), which both have by average a high monthly charge

In [58]:
melted_df = df.loc[df.internetservice!='No', ['customerid']+internet_services[1:]+['monthlycharges']]
melted_df = melted_df.melt(id_vars=['customerid','monthlycharges'], var_name='service', value_name='is_subscribed')
melted_df[melted_df.is_subscribed=='Yes'].groupby(['service']).mean('monthlycharges')

,monthlycharges
service,
deviceprotection,84.822254
onlinebackup,83.081597
onlinesecurity,78.838732
streamingmovies,88.475714
streamingtv,88.736738
techsupport,80.680137


this table shows that streaming services have highest charge relative to other services

## Results

To sum up the results:
* Having a partner is a positive factor that helps avoid customer churn
* Having dependents is also a positive factor, when a customer has dependets, it is less likely to churn
* Probability of churning is approximately double for senior citizens

* As tenure increase the chance that the customer will leave decreases.
* Expect a churn from customers who prefer paperlessbilling or electronic check more than others
* The longer the length of the contract is, the longer the tenure is, consequently, the less probable a customer will leave

* Tech Support and online security services are two crucial services for customer retention, especially for seniors citizens
* Fiber optic internet service has almost double churn percentage than DSL service customers, especially when for those not subscribed in security and support
* Leaving customers have higher average monthly charge as they subscribe to streaming services, so a discount for these services may keep some customers